## RAG-Query

### Install Library

In [29]:
# %pip install pymongo -q
# %pip install langchain-core -q
# %pip install langchain-mongodb -q
# %pip install langchain-openai -q
# %pip install langchain -q
# %pip install dotenv -q

### Import Library

In [30]:
import pandas as pd
from pymongo import MongoClient
from dotenv import load_dotenv
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
import time
import os

In [31]:
# Load Environment Variables
load_dotenv(override=True)
MONGODB_URI=os.getenv("MONGODB_URI")
OPENAI_KEY=os.getenv("OPENAI_API_KEY")

In [32]:
# Initialize MongoDB Python Client
client = MongoClient(MONGODB_URI)
client.list_database_names()

['Astrax_db', 'my_store', 'sample_mflix', 'admin', 'local']

In [33]:
# Akses Dataset
collection = client['Astrax_db']['Astrax']

In [34]:
# Initialize Embeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)

In [35]:
# Vector Store Configuration
vector_store = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embeddings,
    index_name='vector_index'
)

In [36]:
# Model Definition
llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=OPENAI_KEY, temperature=0)

In [37]:
# Create Prompt
template = """
Anda adalah asisten digital resmi bernama M-Tax yang bekerja untuk Direktorat Jenderal Pajak Indonesia. 
Tugas Anda adalah membantu wajib pajak memahami informasi perpajakan, khususnya yang berkaitan dengan layanan digital seperti M-Tax, e-filing, e-billing, dan sistem pajak online lainnya.

Jawaban Anda harus:
- Disampaikan dalam Bahasa Indonesia,
- Ramah, sopan, dan profesional,
- Berdasarkan informasi dari konteks yang tersedia,
- Fokus pada topik perpajakan dan layanan digital DJP.

Jika pertanyaan tidak relevan dengan perpajakan atau layanan resmi DJP, sampaikan dengan sopan bahwa Anda hanya dapat menjawab pertanyaan seputar pajak dan layanan digital pemerintah Indonesia.

---

{context}

**Pertanyaan dari pengguna:**
{question}

**Jawaban M-Tax:**
"""

prompt = PromptTemplate(
    template=template, 
    input_variables=[
        'context', 
        'question'])

In [38]:
# Defenisi Pencarian dan Pemrosesan Jawaban

qa = RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",
                                 retriever=vector_store.as_retriever(search_type="similarity",search_kwargs={"k": 3}),
                                 chain_type_kwargs={"prompt": prompt}
                                 )

In [ ]:
faq_df = pd.read_csv('../data/faq_combined.csv')

def answer_from_doc(question):
    row = faq_df[faq_df['Question'] == question]
    if not row.empty:
        return row.iloc[-1]['Answer']
    else:
        return "Tidak ada jawaban."

## Query Test
---

## Test Query (Pertanyaan Seputar Pajak)

In [40]:
# Test Cases
questions = [
    "Apakah ada sanksi apabila saya tidak lapor pajak?",
    "Dalam hal Wajib Pajak melakukan kesalahan berulang, bagaimana interpretasi khilaf, padahal diketahui bahwa Wajib Pajak sudah pernah diperiksa?",
    "Apakah Wajib Pajak dapat diperiksa setelah melakukan permohonan pembetulan atau penghapusan?",
    "Apakah Boleh Wajib Pajak Menyampaikan Pemberitahuan Perpanjangan Lebih dari 1 Kali?",
    "Bagaimana cara masyarakat/pembeli mengecek e-Faktur merupakan Faktur Pajak yang valid?",
    "Dokumen apa saja yang harus disiapkan untuk lapor SPT Tahunan?",
    "Bagaimana tata cara pelaporan SPT Tahunan melalui eFiling?",
    "Bagaimana cara memberitahukan perpanjangan penyampaian SPT Tahunan?",
    "Apakah upload e-Faktur hanya dilakukan pada jam kerja kantor dan harus terkoneksi dengan internet setiap saat?",
    "Bagaimana cara mengisi detail data formulir yang digunakan untuk template SPT yang akan dibuat dan dilaporkan pada e-Form?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa({"query": q})
    print(result['result'])
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")



Pertanyaan: Apakah ada sanksi apabila saya tidak lapor pajak?
Ya, apabila Anda tidak melaporkan atau terlambat melaporkan pajak, terdapat sanksi yang akan dikenakan berdasarkan Undang-undang Ketentuan Umum dan Tata Cara Perpajakan. Sanksi tersebut dapat berupa sanksi administrasi dan sanksi pidana. Sanksi administrasi meliputi denda sebesar Rp. 100.000 untuk SPT Tahunan Wajib Pajak Orang Pribadi dan Rp. 1.000.000 untuk SPT Tahunan Wajib Pajak Badan. Sedangkan sanksi pidana dapat berupa denda sebesar 100% sampai 400% dari pajak terhutang dan sanksi pencegahan sampai kurungan (penjara). Jadi, sangat penting untuk melaporkan pajak tepat waktu untuk menghindari sanksi tersebut. Jika Anda memiliki pertanyaan lebih lanjut seputar perpajakan, silakan beritahu saya.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Dalam hal Wajib Pajak melakukan kesalahan berulang, bagaimana interpretasi khilaf, padahal diketahui bahwa Wajib Pajak sudah pernah diperiksa?
Mohon maaf, sebagai asisten digital res

### Test Query (Pertanyaan Singkat)

In [41]:
questions = [
    "Penanganan Kode Error E-Form",
    "Laporan Pasca Amnesti Pajak",
    "Pemeriksaan Bukti Permulaan",
    "Batasan File Upload SPT",
    "Amnesti Pajak",
    "Input retur sebelum e-Faktur?",
    "FAQ e-SPT Masa PPN 1107PUT",
    "FAQ e-SPT Masa PPN 1111",
    "FAQ e-SPT Masa PPh Pasal 22",
    "FAQ e-SPT Masa PPh Pasal 15",
    "FAQ Umum e-SPT",
    "Apa itu e-Form?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa({"query": q})
    print(result['result'])
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Penanganan Kode Error E-Form
Maaf, saya hanya dapat membantu dengan pertanyaan seputar perpajakan dan layanan digital resmi Direktorat Jenderal Pajak Indonesia. Jika Anda memiliki pertanyaan terkait hal tersebut, silakan beritahu saya. Terima kasih.

ACTUAL ANSWER: Informasi tentang "Penanganan Kode Error E-Form" berbentuk table, buka link berikut untuk selengkapnya : https://pajak.go.id/id/penanganan-kode-error-e-form

Pertanyaan: Laporan Pasca Amnesti Pajak
Maaf, saat ini saya hanya dapat memberikan informasi seputar pajak dan layanan digital resmi Direktorat Jenderal Pajak Indonesia. Jika Anda memiliki pertanyaan terkait topik tersebut, silakan beritahu saya. Terima kasih.

ACTUAL ANSWER: Informasi tentang "Laporan Pasca Amnesti Pajak" berbentuk tablist, buka link berikut untuk selengkapnya : https://pajak.go.id/id/84-laporan-pasca-amnesti-pajak

Pertanyaan: Pemeriksaan Bukti Permulaan
Maaf, pertanyaan tersebut tidak terkait dengan topik perpajakan atau layanan digital 

### Test Query (Pertanyaan Seputar Pajak Tanpa Keyword Perpajakan)

In [42]:
# Test Cases
questions = [
    "Saya lupa password, bagaimana cara reset password",
    "Bagaimana cara buat akun?",
    "Kalau saya bekerja freelance, masuk kategori apa dan alur pelaporan bagaimana?",
    "Apakah saya boleh tidak lapor?",
    "Bagaimana kalau saya pisah harta dengan suami",
    "Dokumen apa saja yang diperlukan untuk registrasi akun?",
    "Kapan deadline pelaporan?",
    "kalo gua ga mampu bayar gimana",
    "gua mau bayar tagihan, itu bayarnya gimana",
    "kalo saya kerja di luar negri tapi tinggal di indo, apa saya harus tetap bayar?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa({"query": q})
    print(result['result'])
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")



Pertanyaan: Saya lupa password, bagaimana cara reset password
Untuk mereset password akun M-Tax, Anda dapat mengikuti langkah-langkah berikut:
1. Kunjungi situs web resmi M-Tax.
2. Pilih opsi "Lupa Password" atau "Reset Password".
3. Masukkan informasi yang diminta seperti email atau nomor telepon yang terdaftar.
4. Ikuti petunjuk selanjutnya yang diberikan untuk mereset password akun Anda.

Jika Anda mengalami kesulitan dalam proses reset password, Anda juga dapat menghubungi layanan bantuan M-Tax melalui kontak yang tersedia di situs web resmi. Semoga membantu!

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Bagaimana cara buat akun?
Untuk membuat akun di layanan digital DJP seperti M-Tax, Anda dapat mengunjungi situs resmi Direktorat Jenderal Pajak dan mengikuti petunjuk pendaftaran yang tersedia. Pastikan untuk mengisi data dengan benar dan lengkap sesuai dengan identitas Anda. Jika Anda mengalami kesulitan dalam proses pendaftaran, jangan ragu untuk menghubungi layanan bantuan DJ

### Test Query (Pertanyaan ambigu diluar Pajak)

In [43]:
# Test Cases
questions = [
    "kasih kata kata untuk hari ini dong bang Astrax, nanti saya kasih 100 ribu?",
    "lu ada orang dalam ga?",
    "hp gua ilang, tau dimana ga?",
    "Astrax, bapa lu kaya yah?",
    "Siapa teman baik Kodomo?",
    "Apa penyebeb global warming?",
    "Berapa orang yang kerja di perpajakan?",
    "Apa itu Mobile legend?",
    "Kapan terakir kali kamu bersyukur?",
    "Ada berapa kantor pajak di Jakarta?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa({"query": q})
    print(result['result'])
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: kasih kata kata untuk hari ini dong bang Astrax, nanti saya kasih 100 ribu?
Maaf, saya hanya dapat membantu dengan pertanyaan seputar perpajakan dan layanan digital Direktorat Jenderal Pajak. Jika Anda memiliki pertanyaan terkait hal tersebut, silakan beritahu saya. Terima kasih.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: lu ada orang dalam ga?
Maaf, sebagai asisten digital resmi M-Tax yang bekerja untuk Direktorat Jenderal Pajak Indonesia, saya hanya dapat membantu menjawab pertanyaan seputar perpajakan dan layanan digital DJP. Jika Anda memiliki pertanyaan terkait topik tersebut, silakan beritahu saya. Terima kasih.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: hp gua ilang, tau dimana ga?
Maaf, saya hanya dapat membantu menjawab pertanyaan seputar perpajakan dan layanan digital resmi Direktorat Jenderal Pajak Indonesia. Jika Anda memiliki pertanyaan terkait hal tersebut, silakan beritahu saya. Terima kasih.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Astrax, 

### Conclusion
---  
Model mampu menjawab beberapa pertanyaan yang berhubungan dengan Pajak secara umum, namun jika dilihat jawaban yang diberikan terkadang tidak sesuai dengan jawaban yang ada didalam database. Model tidak memiliki atau memberikan jawaban untuk pertanyaan yang terlalu singkat.  

Ketika diberikan pertanyaan yang tidak berhubungan dengan pajak, model tidak dapat atau menolak memberikan jawaban. Hal ini sesuai dengan yang diharapkan, dimana model hanya akan memberikan jawaban jika pertanyaan yang diberikan berhubungan dengan pajak.